<a href="https://colab.research.google.com/github/MickaRiv/ProjetDatascientest-VoiceSeparator/blob/main/streamlit/streamlit_voice-separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prérequis



*   Avoir un compte ngrok sur https://dashboard.ngrok.com
*   récupérer le numéro de token



#Installations des paquets nécessaires

##pip install streamlit

In [1]:
!pip install streamlit

     |████████████████████████████████| 9.9 MB 10.9 MB/s 
     |████████████████████████████████| 181 kB 50.8 MB/s 
     |████████████████████████████████| 4.3 MB 36.6 MB/s 
     |████████████████████████████████| 76 kB 929 kB/s 
     |████████████████████████████████| 111 kB 55.6 MB/s 
     |████████████████████████████████| 164 kB 32.2 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 128 kB 36.1 MB/s 
     |████████████████████████████████| 793 kB 45.6 MB/s 
     |████████████████████████████████| 380 kB 43.6 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=773a3896c3de090cc8db017d596a79f8828ff74022866a82e327ecd03886cc40
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      

##pip install pygnrok

In [2]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 8.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=ebe938d1fbbbe1f3cb5c899054f73a23488ae089ecba39899164a7944631a076
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


## pip install nussl

In [3]:
!pip install nussl

     |████████████████████████████████| 239 kB 7.3 MB/s 
     |████████████████████████████████| 251 kB 49.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 51 kB 48 kB/s 
     |████████████████████████████████| 310 kB 62.7 MB/s 
     |████████████████████████████████| 153 kB 41.0 MB/s 
     |████████████████████████████████| 90 kB 9.7 MB/s 
     |████████████████████████████████| 963 kB 45.8 MB/s 
     |████████████████████████████████| 130 kB 48.0 MB/s 
     |████████████████████████████████| 6.2 MB 21.1 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=dc3a4a08ae21234dd4ca79f694b3222e077522351a22a1e72aa584194add9241
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=8e4ab8cedd782b2f91aee722b450bc2d4f17336c36486dfb4a3522356ad4a2ee
  Stored in d

##pip install pydub

In [4]:
!pip install pydub

##pip install demucs

In [5]:
!pip install demucs

     |████████████████████████████████| 1.5 MB 8.1 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 391 kB 39.4 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
     |████████████████████████████████| 190 kB 28.4 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 70 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 66.8 MB/s 
     |████████████████████████████████| 112 kB 62.2 MB/s 
  Created wheel for demucs: filename=demucs-3.0.4-py3-none-any.whl size=57894 sha256=79c6f5e1440daaa334a16842f4cdc7cc9c47c9db6305af05ad428c498a43dffa
  Stored in directory: /root/.cache/pip/wheels/59/fd/99/0d724080cea2cc254162239a9015bfb69b4277c61dbfb1fe44
  Created wheel for julius: filename=julius-0.2.

##pip install openunmix

In [6]:
!pip install openunmix

##pip install spleeter

## pip install voicesep

In [7]:
!pip install git+https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator

  Cloning https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator to /tmp/pip-req-build-rleihd3s
  Running command git clone -q https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator /tmp/pip-req-build-rleihd3s
  Created wheel for voice-separator: filename=voice_separator-0.0.1-py3-none-any.whl size=7761 sha256=051b55b71b355f441715f0a69e2366fbd8aba36d9855106700ab9cc4534c1f6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5v18usfr/wheels/89/ad/91/9cded351057b1a842fbbfa97f091da426d85036b0efd2e0273
Successfully built voice-separator


# Montage du drive

In [8]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")  # we mount the google drive at /content/drive

Mounted at /content/drive


# Start de ngrok

In [9]:
!ls
!ngrok authtoken 25QRE4uYk7AG52BigElvRGk5TZc_4EfgeeiWXnGXYY2Qf5axB
!ngrok

drive  sample_data
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    n

#st_voice-sepration.py

##Chargement de l'image sur le git

In [10]:
!wget "https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/image.jpg"

--2022-03-09 15:47:18--  https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/image.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11459 (11K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>]  11.19K  --.-KB/s    in 0s      

2022-03-09 15:47:18 (88.3 MB/s) - ‘image.jpg’ saved [11459/11459]



## récupération et exécution de st_voice-sepration.py du git

In [ ]:
!wget https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py

--2022-02-25 14:04:07--  https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6143 (6.0K) [text/plain]
Saving to: ‘st_voice-separation.py’

st_voice-separation 100%[===================>]   6.00K  --.-KB/s    in 0s      

2022-02-25 14:04:07 (42.1 MB/s) - ‘st_voice-separation.py’ saved [6143/6143]



In [ ]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

## exécution du .py dans drive pour éviter les commit push à répétition

In [14]:
!streamlit run --server.port 80 /content/drive/MyDrive/'Projet Datascientest'/Ephi_st/st_voice-separation.py &>/dev/null&

## ou dev direct dans colab

In [ ]:
%%writefile st_voice-separation_from_colab.py
import streamlit as st

st.title("st.title")


Overwriting st_voice-separation.py


In [ ]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

#Lancement du streamlit

In [15]:
from pyngrok import ngrok
# Setup u tunnel to the streamlit port 8501
public_url = ngrok.connect(port='80')
public_url

<NgrokTunnel: "http://d237-35-197-117-17.ngrok.io" -> "http://localhost:80">

#TEST

In [23]:
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import nussl
import os
import numpy as np
import tensorflow as tf
from voicesep.core import get_musdb_data
from voicesep.unet import UNetModel
from pydub import AudioSegment
from os import listdir
from os.path import isfile, isdir, join
import shutil
from tensorflow.keras.models import load_model
from voicesep.demucs import DemucsModel
from voicesep.spleeter import SpleeterModel
from voicesep.openunmix import OpenUnmixModel

#-------------------------------------
# Définition des fonctions 
#-------------------------------------

# Si colab et drive monté
musdb_path = os.path.join("/content","drive","MyDrive","Projet Datascientest","musdb18")
unets_path = os.path.join("/content","drive","MyDrive","Projet Datascientest","UNet")
#-------------------------------------
# Le streamlit 
#-------------------------------------
#%% dev de la sidebar avec chargement du modèle --------------------------


musdb_demo = get_musdb_data(gather_accompaniment=True,folder=musdb_path,subfolder="train")




2022-03-08 17:41:38.382 INFO    pyngrok.process.ngrok: t=2022-03-08T17:41:38+0000 lvl=info msg="join connections" obj=join id=1710f139f302 l=127.0.0.1:80 r=[2a01:e0a:356:aa20:c1d9:adae:56e:b5d1]:52094


In [24]:
print(len(musdb_demo))
for nom_musique in musdb_demo.musdb.tracks:
  numero_musique = musdb_demo.musdb.get_track_indices_by_names(nom_musique.__repr__())
  print(numero_musique,nom_musique)


100
[0] A Classic Education - NightOwl
[1] ANiMAL - Clinic A
[2] ANiMAL - Easy Tiger
[3] ANiMAL - Rockshow
[4] Actions - Devil's Words
[5] Actions - One Minute Smile
[6] Actions - South Of The Water
[7] Aimee Norwich - Child
[8] Alexander Ross - Goodbye Bolero
[9] Alexander Ross - Velvet Curtain
[10] Angela Thomas Wade - Milk Cow Blues
[11] Atlantis Bound - It Was My Fault For Waiting
[12] Auctioneer - Our Future Faces
[13] AvaLuna - Waterduct
[14] BigTroubles - Phantom
[15] Bill Chudziak - Children Of No-one
[16] Black Bloc - If You Want Success
[17] Celestial Shore - Die For Us
[18] Chris Durban - Celebrate
[19] Clara Berry And Wooldog - Air Traffic
[20] Clara Berry And Wooldog - Stella
[21] Clara Berry And Wooldog - Waltz For My Victims
[22] Cnoc An Tursa - Bannockburn
[23] Creepoid - OldTree
[24] Dark Ride - Burning Bridges
[25] Dreamers Of The Ghetto - Heavy Love
[26] Drumtracks - Ghost Bitch
[27] Faces On Film - Waiting For Ga
[28] Fergessen - Back From The Start
[29] Fergessen -

# Arrêt de streamlit

Récupération de l'ID du process

In [13]:
!ps -eaf | grep streamlit

root        1323      67  0 16:23 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root        1325    1323  0 16:23 ?        00:00:00 grep streamlit


Kill du process

In [44]:
!kill 3668

Arrêt de ngrok

In [45]:
ngrok.kill()

2022-03-08 19:11:55.838 INFO    pyngrok.process: Killing ngrok process: 806
